## Predictive Models

In this notebook, we will try multiple methods to predict A&E demands

In [1]:
#!pip install dask[dataframe]
#!pip install graphviz
#!pip install --upgrade pip
#!pip install dask-ml 


In [2]:
import pandas as pd
import geopandas as gpd
import dask.dataframe as dd
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [3]:
url1 = 'https://www.opendata.nhs.scot/dataset/0d57311a-db66-4eaa-bd6d-cc622b6cbdfa/resource/a5f7ca94-c810-41b5-a7c9-25c18d43e5a4/download/weekly_ae_activity_20240714.csv'
df_week_AE = pd.read_csv(url1)

url2 = 'https://www.opendata.nhs.scot/dataset/997acaa5-afe0-49d9-b333-dcf84584603d/resource/022c3b27-6a58-48dc-8038-8f1f93bb0e78/download/opendata_monthly_ae_when_202405.csv'
df_month_AE = pd.read_csv(url2)

url3 = 'https://www.opendata.nhs.scot/dataset/997acaa5-afe0-49d9-b333-dcf84584603d/resource/c4622324-f59c-4011-a67b-83b59c59ca94/download/opendata_monthly_ae_discharge_202405.csv'
df_discharge = pd.read_csv(url3)

url4= 'https://www.opendata.nhs.scot/dataset/997acaa5-afe0-49d9-b333-dcf84584603d/resource/37ba17b1-c323-492c-87d5-e986aae9ab59/download/monthly_ae_activity_202405.csv'
df_month_attendances= pd.read_csv(url4)

url_hospital = 'https://www.opendata.nhs.scot/dataset/cbd1802e-0e04-4282-88eb-d7bdcfb120f0/resource/c698f450-eeed-41a0-88f7-c1e40a568acc/download/hospitals.csv'
df_hospital = pd.read_csv(url_hospital)
#df_hospital : A list of all NHS hospitals across Scotland and associated geographic information. It should be noted that this list contains all hospitals in Scotland, not only acute hospitals.

url_demographics= 'https://www.opendata.nhs.scot/dataset/997acaa5-afe0-49d9-b333-dcf84584603d/resource/6abbf8e4-e4e0-4a56-a7b9-f7c7b4171ff3/download/opendata_monthly_ae_demographics_202405.csv'
df_demographics= pd.read_csv(url_demographics)

url_multiple_attendance= 'https://www.opendata.nhs.scot/dataset/997acaa5-afe0-49d9-b333-dcf84584603d/resource/0ca3b959-b758-4532-bb55-aa86da28679e/download/opendata_monthly_ae_multiple_attendances_202405.csv'
df_multi_attendance= pd.read_csv(url_multiple_attendance)
#This data resource contains multiple attendances statistics on new and unplanned return attendances at Accident and Emergency (A&E) services across Scotland for the latest 12 month period.

In [4]:
shapefile_path = "SG_NHS_HealthBoards_2019"

#Reading the shapefile into a GeoDataFrame
gdf = gpd.read_file(shapefile_path)

#Information about locations
location_df= gdf[["HBCode", "HBName"]]

In [5]:
df_week_AE.head()

,WeekEndingDate,Country,HBT,TreatmentLocation,DepartmentType,NumberOfAttendancesEpisode,NumberWithin4HoursEpisode,NumberOver4HoursEpisode,PercentageWithin4HoursEpisode,NumberOver8HoursEpisode,PercentageOver8HoursEpisode,NumberOver12HoursEpisode,PercentageOver12HoursEpisode
0,20150222,S92000003,S08000015,A210H,Emergency Department,814,624,190,76.7,21,2.6,2,0.2
1,20150222,S92000003,S08000015,A111H,Emergency Department,1347,1115,232,82.8,31,2.3,2,0.1
2,20150222,S92000003,S08000016,B120H,Emergency Department,517,463,54,89.6,1,0.2,0,0.0
3,20150222,S92000003,S08000017,Y146H,Emergency Department,604,578,26,95.7,0,0.0,0,0.0
4,20150222,S92000003,S08000017,Y144H,Emergency Department,196,185,11,94.4,1,0.5,0,0.0


In [6]:
df_month_AE.head()

,Month,Country,HBT,TreatmentLocation,DepartmentType,Day,Week,Hour,InOut,NumberOfAttendances
0,201801,S92000003,S08000015,A111H,Emergency Department,Friday,Weekday,00:00 to 00:59,Out of Hours,20
1,201801,S92000003,S08000015,A111H,Emergency Department,Friday,Weekday,01:00 to 01:59,Out of Hours,14
2,201801,S92000003,S08000015,A111H,Emergency Department,Friday,Weekday,02:00 to 02:59,Out of Hours,6
3,201801,S92000003,S08000015,A111H,Emergency Department,Friday,Weekday,03:00 to 03:59,Out of Hours,5
4,201801,S92000003,S08000015,A111H,Emergency Department,Friday,Weekday,04:00 to 04:59,Out of Hours,5


In [7]:
df_month_attendances.head()

,Month,Country,HBT,TreatmentLocation,DepartmentType,NumberOfAttendancesAll,NumberWithin4HoursAll,NumberOver4HoursAll,PercentageWithin4HoursAll,NumberOfAttendancesEpisode,...,PercentageWithin4HoursEpisode,PercentageWithin4HoursEpisodeQF,NumberOver8HoursEpisode,NumberOver8HoursEpisodeQF,PercentageOver8HoursEpisode,PercentageOver8HoursEpisodeQF,NumberOver12HoursEpisode,NumberOver12HoursEpisodeQF,PercentageOver12HoursEpisode,PercentageOver12HoursEpisodeQF
0,200707,S92000003,S08000015,A101H,Minor Injury Unit or Other,252,252,0,100.0,NaN,...,NaN,z,NaN,z,NaN,z,NaN,z,NaN,z
1,200707,S92000003,S08000015,A111H,Emergency Department,5414,5290,124,97.7,5414.0,...,97.7,NaN,26.0,NaN,0.5,NaN,24.0,NaN,0.4,NaN
2,200707,S92000003,S08000015,A207H,Minor Injury Unit or Other,92,92,0,100.0,NaN,...,NaN,z,NaN,z,NaN,z,NaN,z,NaN,z
3,200707,S92000003,S08000015,A210H,Emergency Department,3530,3355,175,95.0,3530.0,...,95.0,NaN,3.0,NaN,0.1,NaN,1.0,NaN,0.0,NaN
4,200707,S92000003,S08000016,B103H,Minor Injury Unit or Other,20,20,0,100.0,NaN,...,NaN,z,NaN,z,NaN,z,NaN,z,NaN,z


In [8]:
df_discharge.head()

,Month,Country,HBT,TreatmentLocation,DepartmentType,Age,AgeQF,Discharge,DischargeQF,NumberOfAttendances
0,201801,S92000003,S08000015,A111H,Emergency Department,18-24,NaN,Admission to same Hospital,NaN,85
1,201801,S92000003,S08000015,A111H,Emergency Department,18-24,NaN,Discharged Home or to usual Place of Residence,NaN,386
2,201801,S92000003,S08000015,A111H,Emergency Department,18-24,NaN,Transferred to Other Hospital/Service,NaN,5
3,201801,S92000003,S08000015,A111H,Emergency Department,18-24,NaN,NaN,:,18
4,201801,S92000003,S08000015,A111H,Emergency Department,25-39,NaN,Admission to same Hospital,NaN,206


In [9]:
df_hospital.head()

,HospitalCode,HospitalName,AddressLine1,AddressLine2,AddressLine2QF,AddressLine3,AddressLine3QF,AddressLine4,AddressLine4QF,Postcode,HealthBoard,HSCP,CouncilArea,IntermediateZone,DataZone
0,A101H,Arran War Memorial Hospital,Lamlash,Isle of Arran,NaN,NaN,z,NaN,z,KA278LF,S08000015,S37000020,S12000021,S02002097,S01011176
1,A103H,Ayrshire Central Hospital,Kilwinning Road,Irvine,NaN,NaN,z,NaN,z,KA128SS,S08000015,S37000020,S12000021,S02002105,S01011213
2,A105H,Kirklandside Hospital,Kirklandside,Kilmarnock,NaN,Ayrshire,NaN,NaN,z,KA1 5LH,S08000015,S37000008,S12000008,S02001492,S01007961
3,A110H,Lady Margaret Hospital,College St,Millport,NaN,Isle of Cumbrae,NaN,NaN,z,KA280HF,S08000015,S37000020,S12000021,S02002128,S01011328
4,A111H,University Hospital Crosshouse,Kilmarnock Road,Kilmarnock,NaN,Ayrshire,NaN,NaN,z,KA2 0BE,S08000015,S37000008,S12000008,S02001504,S01008027


In [10]:
df_demographics.head()

,Month,Country,HBT,DepartmentType,Age,AgeQF,Sex,SexQF,Deprivation,DeprivationQF,NumberOfAttendances
0,201801,S92000003,S08000015,Emergency Department,18-24,NaN,Female,NaN,1.0,NaN,158
1,201801,S92000003,S08000015,Emergency Department,18-24,NaN,Female,NaN,2.0,NaN,112
2,201801,S92000003,S08000015,Emergency Department,18-24,NaN,Female,NaN,3.0,NaN,50
3,201801,S92000003,S08000015,Emergency Department,18-24,NaN,Female,NaN,4.0,NaN,39
4,201801,S92000003,S08000015,Emergency Department,18-24,NaN,Female,NaN,5.0,NaN,27


In [11]:
df_multi_attendance.head()

,YearEnd,Country,HBT,DepartmentType,OneAttendance,TwoAttendances,ThreeAttendances,FourAttendances,FivePlusAttendances
0,202405,S92000003,S08000015,Emergency Department,50073,11022,3062,1060,986
1,202405,S92000003,S08000016,Emergency Department,14984,3573,1145,426,465
2,202405,S92000003,S08000017,Emergency Department,22579,5419,1804,681,782
3,202405,S92000003,S08000019,Emergency Department,33852,6435,1766,648,652
4,202405,S92000003,S08000019,Minor Injury Unit or Other,9782,639,68,17,2


**Objective:** To build a model that can predict Discharge outcome (Admission to same Hospital/Discharged Home or to usual Place of Residence/Transferred to Other Hospital/Service) based on the Hour and day of arrival. 

The thought behind this objective is that as seen in the exploratory analysis, the demand is not equally distributed across the health boards, therefore it would be useful to predict the outcome of the emergency visit. For example, if a specific board is more likely to provide transfer of the patient, it could imply lack of resources in that board. Similariliy, a board that admits most patients indicates sufficient capacity to deal with emergency patients. Also, boards with most discharges to residence, may be more efficient in handling their resources. 

The fators to be considered include 

#### Data Preparation:

- Merge datasets to include all relevant features for each record.
- Clean the data by handling missing values and encoding categorical variables.
- Create new features if necessary, such as day and hour transformations.

For training the model, we must first create a combined dataset containg all the relevant featurres. 

In order to save memory for merging, we shall first filter the required columns from each dataset. 

In [12]:
#print(df_demographics.columns)
#print(df_discharge.columns)
#print(df_hospital.columns)
#print(df_month_AE.columns)
#print(df_week_AE.columns)

df_demographics= df_demographics[['Month','HBT', 'DepartmentType', 'Age','Sex','Deprivation','NumberOfAttendances']]
df_discharge= df_discharge[['Month','HBT','Discharge', 'NumberOfAttendances']]
df_month_AE= df_month_AE[['Month','HBT','DepartmentType', 'Day','Week', 'Hour', 'InOut', 'NumberOfAttendances']]

# Filter the DataFrame to include only rows with Month >= 202201
df_demographics= df_demographics[df_demographics['Month'] >= 202301]
df_month_AE= df_month_AE[df_month_AE['Month'] >= 202301]
df_discharge= df_discharge[df_discharge['Month'] >= 202301]

In [13]:
df_discharge.head()

,Month,HBT,Discharge,NumberOfAttendances
59554,202301,S08000015,Admission to same Hospital,24
59555,202301,S08000015,Discharged Home or to usual Place of Residence,246
59556,202301,S08000015,Transferred to Other Hospital/Service,6
59557,202301,S08000015,NaN,28
59558,202301,S08000015,Admission to same Hospital,91


In [14]:
df_month_AE.head()

,Month,HBT,DepartmentType,Day,Week,Hour,InOut,NumberOfAttendances
362549,202301,S08000015,Emergency Department,Friday,Weekday,00:00 to 00:59,Out of Hours,15
362550,202301,S08000015,Emergency Department,Friday,Weekday,01:00 to 01:59,Out of Hours,9
362551,202301,S08000015,Emergency Department,Friday,Weekday,02:00 to 02:59,Out of Hours,2
362552,202301,S08000015,Emergency Department,Friday,Weekday,03:00 to 03:59,Out of Hours,7
362553,202301,S08000015,Emergency Department,Friday,Weekday,04:00 to 04:59,Out of Hours,11


In [15]:
# Merge demographics and discharge data
Demo_and_discharge = pd.merge(df_demographics, df_discharge, on=["Month", "HBT"])
Demo_and_discharge.head()

,Month,HBT,DepartmentType,Age,Sex,Deprivation,NumberOfAttendances_x,Discharge,NumberOfAttendances_y
0,202301,S08000015,Emergency Department,18-24,Female,1.0,113,Admission to same Hospital,24
1,202301,S08000015,Emergency Department,18-24,Female,1.0,113,Discharged Home or to usual Place of Residence,246
2,202301,S08000015,Emergency Department,18-24,Female,1.0,113,Transferred to Other Hospital/Service,6
3,202301,S08000015,Emergency Department,18-24,Female,1.0,113,NaN,28
4,202301,S08000015,Emergency Department,18-24,Female,1.0,113,Admission to same Hospital,91


In [16]:
Demo_and_discharge= Demo_and_discharge.rename(columns={'NumberOfAttendances_y': 'Discharge Frequency', 'NumberOfAttendances_x': 'Demographic frequency'})

# Handle missing values
Demo_and_discharge = Demo_and_discharge.dropna(subset=['Discharge'])
Demo_and_discharge.head()

,Month,HBT,DepartmentType,Age,Sex,Deprivation,Demographic frequency,Discharge,Discharge Frequency
0,202301,S08000015,Emergency Department,18-24,Female,1.0,113,Admission to same Hospital,24
1,202301,S08000015,Emergency Department,18-24,Female,1.0,113,Discharged Home or to usual Place of Residence,246
2,202301,S08000015,Emergency Department,18-24,Female,1.0,113,Transferred to Other Hospital/Service,6
4,202301,S08000015,Emergency Department,18-24,Female,1.0,113,Admission to same Hospital,91
5,202301,S08000015,Emergency Department,18-24,Female,1.0,113,Discharged Home or to usual Place of Residence,545


In [17]:
# Merge the above result with the AE data
Arrival_and_discharge_df = pd.merge(df_discharge, df_month_AE, on=["Month", "HBT"])

In [18]:
Arrival_and_discharge_df = Arrival_and_discharge_df .rename(columns={'NumberOfAttendances_y': 'Arrival frequency', 'NumberOfAttendances_x': 'Discharge frequency'})
Arrival_and_discharge_df  = Arrival_and_discharge_df.dropna(subset=['Discharge'])

#### Feature Engineering:

- Identify key features that are likely to impact the discharge outcome.
- Consider interactions between features, such as combining day and hour into a single feature if it makes sense for your model.


In [19]:
# Encode categorical features
encoder = OneHotEncoder()
categorical_features = encoder.fit_transform(Demo_and_discharge[['DepartmentType', 'Age', 'Sex', 'HBT']])

In [20]:
# Scale numerical features
scaler = StandardScaler()
numerical_features = scaler.fit_transform(Demo_and_discharge[['Deprivation', 'Demographic frequency']])

In [21]:
# Combine features
X = pd.concat([pd.DataFrame(categorical_features.toarray()), pd.DataFrame(numerical_features)], axis=1)
y =Demo_and_discharge['Discharge']

#### Model Training and Selection:

- Choose a classification model suitable for multi-class outcomes.
- Tried models include Random Forest, Gradient Boosting, and Neural Networks.

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Model Training and Evaluation:

- Split the data into training and testing sets.
- Train the model using the training set and evaluate its performance on the test set.
- Use metrics like accuracy, precision, recall, and F1-score to assess model performance.

In [23]:
# Train model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [24]:
# Predict and evaluate
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


                                                precision    recall  f1-score   support

                    Admission to same Hospital       0.21      0.10      0.14     85642
Discharged Home or to usual Place of Residence       0.34      0.71      0.46    115024
                                         Other       0.00      0.00      0.00     26672
         Transferred to Other Hospital/Service       0.21      0.09      0.12     93083

                                      accuracy                           0.31    320421
                                     macro avg       0.19      0.22      0.18    320421
                                  weighted avg       0.24      0.31      0.24    320421

[[ 8699 66780     0 10163]
 [16969 81394     0 16661]
 [ 3372 20152     0  3148]
 [11483 73677     0  7923]]


In [25]:
# Drop rows with missing discharge outcomes
df = Demo_and_discharge.dropna(subset=['Discharge'])

# Features and target
X = df.drop(columns=['Discharge'])
y = df['Discharge']


# Define categorical and numerical columns
categorical_cols = ['DepartmentType', 'Age', 'Sex', 'HBT']
numerical_cols = ['Deprivation', 'Demographic frequency']


# Define transformers for each type of feature
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])


In [26]:
# Combine transformers into a single ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Create a pipeline that first transforms the data and then applies SVM
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', SVC(kernel='linear', C=1, gamma='auto', random_state=42))
])

In [27]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the pipeline
pipeline.fit(X_train, y_train)

In [ ]:
# Make predictions
y_pred = pipeline.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


#### Hyperparameter Tuning:

Use techniques like Grid Search or Random Search to find the best hyperparameters for your model.
Model Deployment:

Once satisfied with the model's performance, prepare it for deployment.

#### A model that predicts the nature of demand